<table>
<tr>
    <td width=25%>
        <img src="images/cytech_logo.png">
    </td>
    <td>
        <center>
            <h1>Deep Learning pour NLP 1</h1>
        </center>
    </td>
    <td width=18%>
        Crée par Yann Vernaz <br/>
        Modifié par Yannick Le Nir
    </td>
</tr>
</table>

<br/>
<div id="top"></div>
<center>
    <a style="font-size: 20pt; font-weight: bold">Embeddings</a>
</center>
<br/>

---

Dans ce <i>Tp</i> nous introduisons la représentation des mots par <i>Word Embeddings</i>.

Alors que les vecteurs obtenus par un encodage <i>one-hot</i> sont binaires (0 ou 1), clairsemés (principalement des zéros) et de très grande dimension (même dimension que le nombre de mots du vocabulaire, par example 50 000), les méthodes du type <i>Word Embeddings</i> permettent de construire des vecteurs dense en petite dimension. 

Ces vecteurs denses sont appris à partir des données et contiennent plus d'informations dans un vecteur de dimensions bien moindre (typiquement 50, 100 ou 300).

<img src="images/word_embeddings.jpg" width="50%">

Les méthodes de type <i>embeddings</i> sont donc une famille de techniques de traitement du langage naturel visant à intégrer le sens sémantique dans un espace géométrique. Cela se fait en associant un vecteur numérique à chaque mot d'un dictionnaire, de telle sorte que la distance (par exemple, la distance $\ell_2$ ou la distance cosinus) entre deux vecteurs quelconques capturerait une partie de la relation sémantique entre les deux mots associés.

En convertissant les mots en vecteurs, nous construisons des relations entre les mots. Plus les mots d'une dimension sont similaires, plus ils sont proches. Par exemple :

$${\bf v}(\text{vert}) = [1.2, 0.98, 0.45, ...]\text{ et }{\bf v}(\text{rouge}) = [1.1, 0.2, 0.5, ...]$$

Les valeurs vectorielles de <i>vert</i> et <i>rouge</i> sont très similaires dans une dimension car ce sont toutes les deux des couleurs. La valeur de la deuxième dimension est très différente, car le rouge peut représenter un élément négatif dans les données d'apprentissage, alors que le vert représente la positivité, etc.

En vectorisant, nous construisons différents types de relations entre les mots.

En pratique on utilise deux méthodes :

* Apprendre les vecteurs pour le problème que l'on veut résoudre (par exemple la classification de documents). Dans ce cas on construit un modèle **Word2Vec** en apprenant les poids d'un réseau de neurones.<br/><br/>

* On utilise un modèle (<i>embeddings</i>) pré-entraîné pour résoudre son problème spécifique.

In [ ]:
import os
import pickle
import numpy as np
import gensim
print("Gensim ", gensim.__version__)

Lecture des données (articles de blog).

In [ ]:
DATAPATH = "posts/"
!ls {DATAPATH}

In [ ]:
male_posts,female_post = [],[]

with open(os.path.join(DATAPATH,"male_blog_list.txt"),"rb") as male_file:
    male_posts= pickle.load(male_file)
    
with open(os.path.join(DATAPATH,"female_blog_list.txt"),"rb") as female_file:
    female_posts = pickle.load(female_file)

print("Nombre de posts pour les hommes: ", len(female_posts))
print("Nombre de posts pour les femmes: ", len(male_posts))

In [ ]:
# on élimine les posts qui ne contiennent aucun mot.
filtered_male_posts = list(filter(lambda p: len(p) > 0, male_posts))
filtered_female_posts = list(filter(lambda p: len(p) > 0, female_posts))

print("Nombre de posts (filtrés) pour les hommes: ", len(filtered_female_posts))
print("Nombre de posts (filtrés) pour les femmes: ", len(filtered_male_posts))

posts = filtered_female_posts + filtered_male_posts

In [ ]:
posts[100]

Pré-traitement des données.

In [ ]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def preprocess_text( raw_post ):
    
    # 1. supprimer les caractères qui ne sont pas des lettres      
    letters_only = re.sub("[^a-zA-Z]", " ", raw_post) 
    
    # 2. convertir en minuscules + split des mots
    words = letters_only.lower().split()                             
    
    # 3. ensemble des "stops words" (article, ...)
    stops = set(stopwords.words("english"))                  
    
    # 5. supprimer les "stop words"
    meaningful_words = [w for w in words if not w in stops]
    
    # 6. on retourne une chaîne séparée par un espace.
    return( " ".join( meaningful_words )) 

preprocess_posts = preprocess_text( posts[100] )
print("Post original : \n", posts[100])
print("\nPost pré-traité : \n", preprocess_posts)

_Tokenisation_ des posts.

In [ ]:
from nltk import word_tokenize

num_posts = len(posts)
clean_posts = []

for i in range( 0, num_posts ):
    clean_posts.append(word_tokenize(preprocess_text( posts[i])))

In [ ]:
clean_posts[100]

# Apprentissage Word2Vec

Créons un modèle _Word2Vec_ en utilisant la librairie [`Gensim`](https://radimrehurek.com/gensim/).

In [ ]:
?gensim.models.Word2Vec

In [ ]:
# sg : training algorithm, 1 for skip-gram; 0 for CBOW.

model_word2vec = gensim.models.Word2Vec(clean_posts, vector_size=50,sg=1, window=10, min_count=1, workers=4)

In [ ]:
# persistence du modèle

model_word2vec.save("model_word2vec.bin")
#model_word2vec = Word2Vec.load("model_word2vec.bin")

In [ ]:
?model_word2vec

In [ ]:
# taille embeddings
model_word2vec.wv.vectors.shape

In [ ]:
print("Taille du vocabulaire: {0} mots.".format(len(model_word2vec.wv.index_to_key)))

# liste de mots du vocabulaire
model_word2vec.wv.index_to_key 

In [ ]:
# vecteur (embeddings) pour le mot 'computer'
vec_computer = model_word2vec.wv["computer"] 

print("Taille du vecteur: {0}\n".format(vec_computer.shape[0]))
print("Vecteur pour le mot 'computer'\n\n {0}".format(vec_computer))

Calculons la similarité (cosinus) entre deux mots.

In [ ]:
?model_word2vec.wv.similarity

In [ ]:
model_word2vec.wv.similarity("woman", "man")

In [ ]:
model_word2vec.wv.similarity("woman", "football")

In [ ]:
model_word2vec.wv.similarity("ring", "husband")

In [ ]:
model_word2vec.wv.similarity("women", "housewife") 

In [ ]:
model_word2vec.wv.similarity("men", "stupid") 

In [ ]:
model_word2vec.wv.similarity("women", "stupid") 

À présent nous allons chercher les mots qui se ressemblent le plus.

In [ ]:
?model_word2vec.wv.most_similar

In [ ]:
model_word2vec.wv.similar_by_word("cat", topn=3)

Effectuons l'analogie suivante: ${\bf v}(\text{king}) - {\bf v}(\text{man}) + {\bf v}(\text{woman})$

In [ ]:
model_word2vec.wv.most_similar(
    positive=["king", "woman"], negative=["man"], topn=3)

Quel mot de la liste ne correspond pas aux autres ?

In [ ]:
?model_word2vec.wv.doesnt_match

In [ ]:
model_word2vec.wv.doesnt_match("breakfast car dinner lunch".split())

# Modèle pré-entraîné

In [ ]:
import gensim.downloader as api

info = api.info()

#pretrained_word2vec = api.load("glove-wiki-gigaword-50")
pretrained_word2vec = api.load("glove-wiki-gigaword-100")
#pretrained2_word2vec = api.load("word2vec-google-news-300")

<hr style="height:3px;border-top:1px solid #fff" />

> **EXERCICE 1**
>
> Effectuer les mêmes tests que précédemment en utilisant le modèle pré-entraîné. 
>
> Conclure ?

<hr style="height:3px;border-top:1px solid #fff" />

In [ ]:
## Votre code ici
## ...

# Keras - Word Embeddings

In [ ]:
import os
import io
import shutil
import re
import string

%matplotlib inline
from matplotlib import pyplot as plt

import tensorflow as tf
print("TensorFlow", tf.__version__)

Nous allons construire un modèle pour prédire les sentiments (positif vs. négatif) de commentaires de films (IMDB). Nous limitons le corpus aux 10 000 mots les plus communs et coupons les commentaires après 100 mots.

In [ ]:
# télécharger le jeu de données IMDb
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')

In [ ]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

In [ ]:
# Le répertoire `train` comporte des dossiers supplémentaires qui doivent être supprimés 
# avant de créer nos ensembles de données.
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Nous utilisons le répertoire `train` pour créer les ensembles d'entrainement (80%) et de validation (20%).

In [ ]:
BATCH_SIZE = 1024
SEED = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=BATCH_SIZE, validation_split=0.2, 
    subset='training', seed=SEED)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=BATCH_SIZE, validation_split=0.2, 
    subset='validation', seed=SEED)

In [ ]:
# Regardons quelques critiques de films et leurs labels (1: positif, 0: négatif).
for text_batch, label_batch in train_ds.take(1):
    for i in range(2):
        print(label_batch[i].numpy(), text_batch.numpy()[i])
        print("\n")

#### NOTE  - Performance

Nous utilisons deux méthodes importantes lors du chargement des données pour nous assurer que les entrées/sorties ne deviennent pas bloquantes :

- `.cache()` conserve les données en mémoire après leur chargement hors disque. Cela permet de s'assurer que l'ensemble des données ne devient pas un goulot d'étranglement lors de l'entrainement du modèle. Si l'ensemble de données est trop important pour tenir en mémoire, nous pouvons utiliser cette méthode pour créer un cache sur le disque (performant et plus efficace que lire de nombreux petits fichiers).<br/><br/>

- La méthode `.prefetch()` superpose le prétraitement des données et l'exécution du modèle pendant l'entrainement.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Couche d'`Embedding`

La librairie `Keras` dispose d'une classe [_Embedding_](https://keras.io/layers/embeddings/) qui transforme des entiers positifs (index des mots) en vecteurs denses de taille fixe. 

La couche d'`Embedding` peut être comprise comme une table de correspondance entre les indices entiers (qui représentent des mots spécifiques) et les vecteurs denses (leurs _embeddings_). La dimensionnalité de l'_embedding_ est un hyper-paramètre que nous devons expérimenter pour voir ce qui fonctionne le mieux sur votre problème.

In [ ]:
# Embedding en 5 dimensions pour un vocabulaire de 1 000 mots.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

Pour les problèmes avec des séquences (phrases), la couche d'_Embedding_ prend un tenseur 2D d'entiers `(samples, sequence_length)`, où chaque entrée est une séquence d'entiers. Les séquences peuvent être de longueurs variables. On peut alimenter la couche d'_embedding_ avec des lots de taille `(32, 10)` (lot de 32 séquences de longueur 10) ou `(64, 15)` (lot de 64 séquences de longueur 15), etc.

Le tenseur en sortie a un axe de plus que l'entrée, les vecteurs d'_Embedding_ sont alignés le long du dernier axe. Si le lot d'entrée a une taille de `(32, 100)`, la sortie est `(32, 100, N)` où `N` est la dimension de l'_Embedding_ (taille du vecteur).

In [ ]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

Une couche d'_Enbedding_ renvoie un tenseur 3D à virgule flottante, de taille `(samples, sequence_length, embedding_dimensionality)`. Pour convertir cette séquence de longueur variable en une représentation fixe, il existe plusieurs approches standard. Vous pouvez utiliser une couche RNN (Recurrent Neural Network), d'_Attention_, ou une couche de _pooling_ avant de la passer à une couche _Dense_. Nous utilisons le _pooling_ car c'est le plus simple.

## Prétraitement du texte

À présent, nous définissons les étapes de prétraitement de l'ensemble des données nécessaires à notre modèle de classification des sentiments. 

In [ ]:
# normalisation
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

# taille du vocabulaire et nombre de mots (max.) dans une séquence
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

# Utilisons la couche TextVectorisation pour normaliser, tokenizer 
# et mapper les chaînes de caractères en nombres entiers. 
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

# Création d'un ensemble de données textuelles (pas d'étiquettes) 
# et utilisation de `adapt` pour construire le vocabulaire.
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Classifieur de sentiments

<hr style="height:3px;border-top:1px solid #fff" />

> **EXERCICE 2**
>
> Construire un réseau de neurones (avec une couche cachée de 16 neurones) pour classifier (<font color='green'>positif</font> / <font color='red'>négatif</font>) les critiques IMDb en utilisant une couche `Embedding` pour calculer des vecteurs denses de notre vocabulaire. 
>
> NOTE : Utiliser une couche `GlobalAveragePooling1D` pour obtenir des séquences de même longueur.

<hr style="height:3px;border-top:1px solid #fff" />

In [ ]:
EMBEDDING_DIM = 16

model = tf.keras.models.Sequential([
    vectorize_layer,
    
    ## Votre code ici
    ## ...
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=15)

In [ ]:
def plot_learning_curves(history):
    acc      = history.history["accuracy"]
    val_acc  = history.history["val_accuracy"]
    loss     = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(len(acc))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    ax1.plot(epochs, acc, "b", label="Précision entraînement")
    ax1.plot(epochs, val_acc, "r", label="Précision validation")
    ax1.set_title("Précision")
    ax1.legend()
    
    ax2.plot(epochs, loss, "b", label="Perte entraînement")
    ax2.plot(epochs, val_loss, "r", label="Perte validation")
    ax2.set_title("Perte")
    ax2.legend()

    fig.show()

plot_learning_curves(history)

## Embedding Projector

Nous allons à présent exporter nos <i>Embeddings</i> pour les visualiser.

Nous allons récupérer les _words embeddings_ appris par le modèle. Les _embeddings_ sont des poids de la couche d'_Embedding_ du modèle. La matrice des poids est de taille `(VOCAB_SIZE, SEQUENCE_LENGTH)`.

On obtient les poids du modèle en utilisant `get_layer()` et `get_weights()`. La fonction `get_vocabulary()` nous fournit le vocabulaire pour construire un fichier de métadonnées avec un _token_ par ligne.

In [ ]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Nous utilisons l'[Embedding Projector](http://projector.tensorflow.org) pour visualiser nos _embeddings_. On doit créer deux fichiers au format tsv (séparation par des tabulations) : un fichier de vecteurs (contenant les _embeddings_) et un fichier de métadonnées (contenant les mots).

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0: continue # skip 0, c'est du "padding".
    vec = weights[index] 
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

Ouvrez l'[Embedding Projector](http://projector.tensorflow.org) (qui peut également fonctionner dans une instance locale de``TensorBoard`).

 - Cliquez sur "Load Data".
    
 - Téléchargez les deux fichiers que vous avez créés ci-dessus : `vectors.tsv` et `metadata.tsv`.

Les _embeddings_ que vous avez formées s'affichent maintenant. Vous pouvez rechercher des mots pour trouver leurs voisins les plus proches. 

Par exemple, essayez de rechercher "men". Vous verrez peut-être des voisins comme "women".